In [1]:
# Importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

import mlflow
import mlflow.sklearn



# Load the data

In [2]:
# Loading data the wine-quality csv file from the URL

csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
try:
        data = pd.read_csv(csv_url, sep=";")
except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

In [3]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


# Explore the data 

In [4]:
allcolumns = data.columns

unique_counts = []
missing_counts = []

for col in allcolumns:
    unique_counts.append(round((data[col].nunique() * 100)/data.shape[0],3))
    missing_counts.append(round((data[col].isna().sum() * 100)/data.shape[0],3))
    

dt = {'Feature': allcolumns, '% of unique values': unique_counts, '% of Missing values': missing_counts}
data_counts  = pd.DataFrame.from_dict(dt)

data_counts.sort_values(by='% of Missing values', ascending=False)

,Feature,% of unique values,% of Missing values
0,fixed acidity,6.004,0.0
1,volatile acidity,8.943,0.0
2,citric acid,5.003,0.0
3,residual sugar,5.691,0.0
4,chlorides,9.568,0.0
5,free sulfur dioxide,3.752,0.0
6,total sulfur dioxide,9.006,0.0
7,density,27.267,0.0
8,pH,5.566,0.0
9,sulphates,6.004,0.0


# Labeling quality class: 1 = good , 0 = bad
Binary Classification

In [5]:
data['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [6]:
data['quality'] = [1 if i > 6.5 else 0 for i in data['quality']]

In [7]:
data['quality'].value_counts()

0    1382
1     217
Name: quality, dtype: int64

# Data Splitting 

In [8]:
X = data.drop(columns = 'quality')
y = data[['quality']]

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)


In [9]:
# Running MLFlow script
with mlflow.start_run():
# Build model using grid search and cross fold validation and declare hyperparameters to tune
        
    parameters = {'randomforestclassifier__n_estimators': [10, 50, 100,200], 
                  'randomforestclassifier__criterion': ['gini', 'entropy'], 'randomforestclassifier__max_depth': [3, 5, 10]}
    metrics = ['f1', 'recall', 'precision', 'roc_auc', 'neg_log_loss', 'neg_brier_score', 
           'average_precision', 'balanced_accuracy']
    pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestClassifier(max_depth=2, random_state=0))
 
    clf = GridSearchCV(pipeline, parameters, scoring=metrics, refit='f1',n_jobs = -1)
    clf.fit(X_train, y_train)
    
    # Logging params and metrics to MLFlow
    score = clf.score(X_train, y_train)
    print("Score: %s" % score)
    mlflow.log_metric("score", score)
    
    for k in clf.best_params_.keys():
         mlflow.log_param(k, clf.best_params_[k])
            
    # Logging model to MLFlow
    mlflow.sklearn.log_model(clf.best_estimator_, "best model")
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
    
    

C:\Users\Rozita\anaconda3\envs\deploy\lib\site-packages\sklearn\pipeline.py:382: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Score: 0.9794721407624634
Model saved in run 20c925e9996348f39bb45ac1ba71a2cb


C:\Users\Rozita\anaconda3\envs\deploy\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
